In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.feature import RegexTokenizer, Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from nltk.classify import NaiveBayesClassifier
from toxic_funcs import Cleaner

In [2]:
# Start Spark session
spark = SparkSession.builder.appName('toxic').getOrCreate()
# Using directly pyspark is giving rubbish so creating pandas df and converting to pyspark dataframe
sqlCtx = SQLContext(spark)
# Pandas dataframe
panda_df = pd.read_csv("data/train.csv")
# Converting pandas to pyspark
pyspark_dataframe = sqlCtx.createDataFrame(panda_df)
pyspark_dataframe.select("comment_text").show(100)
pyspark_dataframe.show()

+--------------------+
|        comment_text|
+--------------------+
|Explanation
Why ...|
|D'aww! He matches...|
|Hey man, I'm real...|
|"
More
I can't ...|
|You, sir, are my ...|
|"

Congratulati...|
|COCKSUCKER BEFORE...|
|Your vandalism to...|
|Sorry if the word...|
|alignment on this...|
|"
Fair use ratio...|
|bbq 

be a man ...|
|Hey... what is it...|
|Before you start ...|
|Oh, and the girl ...|
|"

Juelz Santan...|
|Bye! 

Don't lo...|
|REDIRECT Talk:Voy...|
|The Mitsurugi poi...|
|Don't mean to bot...|
|"

 Regarding y...|
|"
Good to know. ...|
|"

 Snowflakes ...|
|"

 The Signpos...|
|"

Re-consideri...|
...|ial symmetry 
|There's no need t...|
|Yes, because the ...|
|"
Ok. But it wil...|
|"== A barnstar fo...|
|How could I post ...|
|Not sure about a ...|
|Praise 

looked...|
|I was able to pos...|
|"
Well, not ""be...|
|"

Not at all, ...|
|"

 ""Mainland ...|
|pretty much every...|
|Hi Explicit, can ...|
|Notability of Rur...|
|"
 Sure, but the...|
|TFD 

I think w...|
|Y

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

C:\Users\mrbal\Anaconda3\envs\py36\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mrbal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
sid = SentimentIntensityAnalyzer()

In [6]:
comments = pyspark_dataframe.select("comment_text").collect()
#Convert the dataframe column to list
comment_list = list(
    pyspark_dataframe.select('comment_text').toPandas()['comment_text']
)

Py4JJavaError: An error occurred while calling o54.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 3.0 failed 1 times, most recent failure: Lost task 3.0 in stage 3.0 (TID 9, localhost, executor driver): org.apache.spark.SparkException: Python worker did not connect back in time
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:143)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:72)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:117)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:86)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:63)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:324)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:288)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpo

In [ ]:
text_data = pyspark_dataframe.select('comment_text').show()

In [13]:
spark.stop()